# [Spring 2022] CS 4361 / 5361 - Ensembles

## **Before you start**

Make a copy of this Colab by clicking on File > Save a Copy in Drive

After making a copy, add your student id, last name, and first name to the title.

Credit: This notebook was originally created by Dr. Olac Fuentes. It was adapted by Dr. Diego Aguirre on 02/28/2022


In [ ]:
student_name = "Ricardo Sillas"
student_id = "88613474"
print("Identifying Information: ",student_name+","+student_id); # Don't change this line.

Identifying Information:  Ricardo Sillas,88613474


In this assignment, we will be predicting the running times of GPU operations under various parameter settings using ensembles of trees.


Let's load the libraries we will be using.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.metrics import accuracy_score, confusion_matrix,mean_squared_error,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from google.colab import files

Let's upload the dataset we will be working with (you'll find the file on Blackboard).

In [ ]:
from google.colab import files
import os

if not os.path.isfile("gpu_running_time.csv"):
  files.upload() # If this doesn't work, try using Google Chrome

Saving gpu_running_time.csv to gpu_running_time.csv


In [ ]:
df = pd.read_csv('gpu_running_time.csv')
df

,MWG,NWG,KWG,MDIMC,NDIMC,MDIMA,NDIMB,KWI,VWM,VWN,STRM,STRN,SA,SB,Run1 (ms),Run2 (ms),Run3 (ms),Run4 (ms)
0,128,128,16,16,32,32,32,8,2,4,1,0,1,1,13.29,13.25,13.36,13.37
1,128,128,16,16,32,32,32,8,2,4,1,1,1,1,13.29,13.36,13.38,13.65
2,128,128,16,16,32,32,32,2,2,2,1,1,1,1,13.78,13.76,13.73,13.69
3,128,128,16,16,32,32,32,8,2,2,1,1,1,1,14.34,14.44,14.43,14.58
4,128,64,16,16,16,16,32,2,2,2,1,1,1,1,14.61,14.69,14.80,14.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241595,128,128,32,8,8,16,32,8,2,2,0,0,1,1,3322.83,3313.44,3359.22,3342.30
241596,128,128,32,8,8,32,8,8,2,2,0,0,1,1,3324.15,3324.11,3332.74,3300.80
241597,128,128,32,8,8,16,16,8,2,2,0,0,1,1,3325.87,3340.98,3333.41,3341.08
241598,128,128,32,8,8,32,16,8,2,2,0,0,1,1,3333.92,3335.08,3354.68,3317.04


We convert the dataframe to a numpy array and extract the features (X) and target (y). 



In [ ]:
data = df.to_numpy()
X = data[:,:15]
y = np.mean(data[:,15:],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4361)

As a baseline, we evaluate the error when we use the mean y train as predicition for all examples in the test set. 

In [ ]:
pred = np.mean(y_train)+np.zeros_like(y_test)
print('Mean squared error = {:5.2f}'.format(mean_squared_error(pred,y_test)))
print('Mean absolute error =  {:5.2f}'.format(mean_absolute_error(pred,y_test)))

Mean squared error = 134935.89
Mean absolute error =  216.03


Now let's evaluate a decision tree regressor for this task. 

In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train,y_train)
pred = model.predict(X_test)
print('Mean squared error = {:5.2f}'.format(mean_squared_error(pred,y_test)))
print('Mean absolute error =  {:5.2f}'.format(mean_absolute_error(pred,y_test)))

Mean squared error =  8.98
Mean absolute error =   1.05


We'd expect a random forest to perform better than a single tree. Let's see if that is the case.

In [ ]:
model = RandomForestRegressor()
model.fit(X_train,y_train)
pred = model.predict(X_test)
print('Mean squared error = {:5.2f}'.format(mean_squared_error(pred,y_test)))
print('Mean absolute error =  {:5.2f}'.format(mean_absolute_error(pred,y_test)))

Mean squared error =  4.89
Mean absolute error =   0.80


k-nn can also be used for regression. Let's see how well it works in this problem. Warning: it takes a while to run!

In [ ]:
model = KNeighborsRegressor(algorithm='brute',n_jobs=-1)
model.fit(X_train,y_train)
pred = model.predict(X_test)
print('Mean squared error = {:5.2f}'.format(mean_squared_error(pred,y_test)))
print('Mean absolute error =  {:5.2f}'.format(mean_absolute_error(pred,y_test)))

Mean squared error = 11.49
Mean absolute error =   1.63


## Exercises

Let's build our own ensembles of decision tree regressors to see if we can surpass the performance of random forests using the same number of individual trees (the sklearn implementation uses 100 by default). So, instead of using sklearn's RandomForestRegressor, we will build the ensemble ourselves by building 100 trees (use a loop).

1. Build an ensemble that uses 100 trees with random splitting instead of best splitting to solve the problem. Print the mean squared error and mean absolute error evaluated on the test dataset. Each tree should be an instance of the DecisionTreeRegressor class - use the [splitter](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) argument to tell sklearn how to choose the split at each node.

In [ ]:
predictionList = []
finalPredictionList = []

for i in range(100):
  model = DecisionTreeRegressor(splitter = 'random')
  model.fit(X_train,y_train)
  pred = model.predict(X_test)
  predictionList.append(pred)

for i in range(len(predictionList[0])):
  col = [row[i] for row in predictionList]
  mostCommon = max(col)
  finalPredictionList.append(mostCommon)

print('Mean squared error = {:5.2f}'.format(mean_squared_error(finalPredictionList,y_test)))
print('Mean absolute error = {:5.2f}'.format(mean_absolute_error(finalPredictionList,y_test)))

Mean squared error = 36.66
Mean absolute error =  3.01


2. Build an ensemble that uses bagging and 100 trees to solve the problem. Print the mean squared error and mean absolute error evaluated on the test dataset. Each tree should be an instance of the DecisionTreeRegressor class.

In [ ]:
predictionList = []
finalPredictionList = []

for i in range(100):
  a = np.random.randint(low=0, high=X_train.shape[0], size=X_train.shape[0])
  model = DecisionTreeRegressor()
  model.fit(X_train[a],y_train[a])
  pred = model.predict(X_test)
  predictionList.append(pred)

for i in range(len(predictionList[0])):
  col = [row[i] for row in predictionList]
  mostCommon = max(col)
  finalPredictionList.append(mostCommon)

print('Mean squared error = {:5.2f}'.format(mean_squared_error(finalPredictionList,y_test)))
print('Mean absolute error = {:5.2f}'.format(mean_absolute_error(finalPredictionList,y_test)))

Mean squared error = 37.08
Mean absolute error =  3.40


3. Build an ensemble that uses random attribute selection and 100 regression trees to solve the problem. Print the mean squared error and mean absolute error evaluated on the test dataset. Each tree should be an instance of the DecisionTreeRegressor class.

In [ ]:
predictionList = []
finalPredictionList = []

for i in range(10):
  a = np.random.choice(a=[False, True], size=X_train.shape[1])
  model = DecisionTreeRegressor()
  model.fit(X_train[:,a],y_train)
  pred = model.predict(X_test[:,a])
  predictionList.append(pred)

for i in range(len(predictionList[0])):
  col = [row[i] for row in predictionList]
  most_common = max(col, key = col.count)
  finalPredictionList.append(most_common)

print('Mean squared error = {:5.2f}'.format(mean_squared_error(finalPredictionList,y_test)))
print('Mean absolute error = {:5.2f}'.format(mean_absolute_error(finalPredictionList,y_test)))

Mean squared error = 7544.72
Mean absolute error = 24.98


## Submission Instructions

1. File > Download .ipynb
2. Go to Blackboard, find the submission page, and upload the .ipynb file you just downloaded.